# TP4 - TEXT MINING

## Visión General del Notebook

En esta notebook vamos a ver un ejemplo básico de **text minning aplicado a cuentos de Hernán Casciari** que están disponibles en internet. El objetivo de text mining es extraer informacion util de una base de datos de texto (corpus). El corpus que veremos aqui esta compuesto por los cuentos que Casciari fue subiendo a su blog, en forma de posteos, a lo largo de los años.

El ejercicio integra conceptos fundamentales de NLP como:

* Preprocesamiento de texto
* Tokenización
* Eliminación de stop words
* Vectorización de texto
* Análisis de frecuencias
* Visualización de datos textuales

Las librerías fundamentales utilizadas son:

* pandas: Para manipulación y análisis de datos estructurados
* re: Para procesamiento de expresiones regulares
* nltk: Natural Language Toolkit, fundamental en NLP
* sklearn: Para vectorización de texto y otras herramientas de ML
* wordcloud: Para visualización de frecuencias de palabras
* matplotlib: Para visualizaciones generales


Para este ejercicio integrador vamos a basarnos en este [tutorial](https://www.aprendemachinelearning.com/ejercicio-nlp-cuentos-de-hernan-casciari-python-espanol/) donde la primera parte se trataba de **scrapear** el blog para obtener los textos. Sin embargo la url del blog ya no está disponible y los cuentos se pueden adquirir [aquí](https://hernancasciari.com/libros/) junto a otros contenidos super interesantes como sus audiolibros (sí, es un excelente narrador también).

La idea entonces es tratar de ver "**que tenia casciari en su cabeza**" cuando escribio estos cuentos desde 2004 hasta 2015

# Introducción al Text Mining con Cuentos de Hernán Casciari

## Text Mining

El text mining o minería de texto es una técnica que permite extraer información valiosa y patrones significativos de grandes volúmenes de texto no estructurado. En este notebook, realizaremos un análisis de text mining sobre los cuentos del escritor argentino Hernán Casciari, publicados en su blog entre 2004 y 2015.

## Objetivos del Análisis
- Explorar los patrones temáticos en la escritura de Casciari
- Analizar la evolución de su vocabulario a lo largo del tiempo
- Identificar temas recurrentes en sus cuentos
- Visualizar las palabras más significativas por año

## Datos
Los textos analizados provienen de los cuentos publicados en el blog del autor. Aunque originalmente estos textos fueron scrapeados de su blog, actualmente se pueden encontrar en su sitio web oficial (hernancasciari.com/libros/).

## Estructura del Análisis
1. Carga y preparación de datos
2. Limpieza y preprocesamiento de texto
3. Creación de bag of words
4. Análisis de frecuencias
5. Visualizaciones y análisis temporal

### 1.- CARGAMOS LOS DATOS A ANALIZAR

En vez de conseguir los libros scrapeando la página vamos a cargarlos desde la carpeta de la materia. Son 12 archivos pickleados con terminación .txt correspondientes a los cuentos subidos a su blog en los anios de 2004 a 2015.

A pesar de estar en formato .txt los archivos fueron *pickleados* por lo que debemos cargarlos usando la librería pickle.

In [ ]:
# MONTAMOS EL DRIVE PARA CARGAR LOS ARCIVOS
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
path = 'C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\'

In [ ]:
# IMPORTAMOS LAS LIBRERIAS
import pickle # Utilizada para guardar (serializar) y cargar (deserializar) objetos de Python en/desde archivos binarios.
import pandas as pd
pd.set_option('max_colwidth', 150) # Para que las columnas del dataframe muestren hasta 150 caracteres

In [ ]:
# DEFINIMOS UNA LISTA PARA REFERENCIAR LOS AÑOS DE LAS PUBLICACIONES
anios = ['2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015']

In [ ]:
# DEFINIMOS UN DICCIONARIO PARA ALMACENAR DATOS
data = {}

In [ ]:
# CARGAMOS CADA ARCHIVO TXT EN FORMATO BINARIO (SERIALIZADO)
for anio in anios:
    nombre_archivo = path + anio + '.txt'
    try:
        with open(nombre_archivo, "rb") as archivo:
            data[anio] = pickle.load(archivo)
        print(f"Archivo {anio}.txt cargado correctamente")
    except:
        print(f"No se pudo cargar el archivo {anio}.txt")

In [ ]:
# VALIDAMOS LA CARGA DE LOS ARCHIVOS EN EL DICCIONARIO. EN ESTE CASO, VERIFICAMOS LAS KEYS
print(data.keys())

In [ ]:
# VALIDAMOS LOS PRIMEROS 100 CARACTERES PARA CADA UNO
print(data['2004'][0:100])
print(data['2005'][0:100])
print(data['2006'][0:100])
print(data['2007'][0:100])
print(data['2008'][0:100])
print(data['2009'][0:100])
print(data['2010'][0:100])
print(data['2011'][0:100])
print(data['2012'][0:100])
print(data['2013'][0:100])
print(data['2014'][0:100])
print(data['2015'][0:100])

### NOTA DE IMPLEMENTACION DE PYTHON

Al intentar crear un DataFrame utilizando un diccionario donde las llaves son valores escalares (es decir, valores únicos y no iterables, como números o cadenas de texto simples), se produce un error porque pandas espera que cada llave tenga un iterable como valor (como listas o arrays).

Por ejemplo, si tenemos el siguiente diccionario:

data = {
    'fruta1': 'manzana',
    'fruta2': 'banana'
}

Aquí, 'fruta1' y 'fruta2' son claves escalares y sus respectivos valores son cadenas simples. Al intentar crear un DataFrame, obtendremos un error porque pandas no puede interpretar varios registros a partir de este formato.

Para solucionarlo, convertimos los valores del diccionario en listas, creando un nuevo diccionario así:

data_combined = {
    'fruta1': ['manzana'],
    'fruta2': ['banana']
}

Ahora, cada valor es una lista (iterable), lo que permite que pandas construya un DataFrame correctamente.

In [ ]:
# RETORNA UNA VISTA DE TODOS LOS PARES KEY-VALUE DEL DICCIONARIO.
data.items()

In [ ]:
# NUEVO DICCIONARIO PASANDO LOS VALORES DE LAS KEY A LISTAS
data_combined = {key: [value] for (key, value) in data.items()}

# CREAMOS UN DATAFRAME
data_df = pd.DataFrame.from_dict(data_combined).transpose() # transponemos para que tenga las dimensiones correctas
data_df.columns = ['transcript']                            # renombramos las columnas (en este caso 1)
data_df = data_df.sort_index()                              # ordenamos los indices de forma creciente
print(data_df)

### 2.- LIMPIEZA DE DATOS

Ahora aplicaremos algunos de los filtros de limpieza que se suelen usar para poder tratar el texto:

* Pasar texto a minúsculas
* Quitar signos de puntuación (interrogación, y otros símbolos)
* Quitar espacios extra, cambio de carro, tabulaciones

Para eso vamos a ultilizar las librerías "re" y "string"

In [ ]:
# IMPORTAMOS LAS LIBRERIAS QUE NECESITAMOS
import re       # libreria de expresiones regulares
import string   # libreria de cadena de caracteres

RECORDAMOS: Expresiones regulares usan '\\' para formas especiales, ej: '\n' es salto de linea. si queremos imprimir una cadena de caracteres que sea '\\' seguido de 'n' se usa la notacion de cadena *raw* en Python que es poner una 'r' antes de la cadena de caracteres.

Dicho esto, para encontrar coincidencias en strings en python se utiliza la libreria [re](https://docs.python.org/es/3/library/re.html) de expresiones regulares.

Hay muchisimas expresiones y caracteres especiales que sirven para trabajar con texto. Les invito a buscar en la documentacion mas informacion. Vamos a mostrar aca las mas usuales:

  " . ": coincide con cualquier caracter excepto con una nueva línea

  "\*": Hace que el RE resultante coincida con 0 o más repeticiones del RE precedente, tantas repeticiones como sean posibles. ab* coincidirá con “a”, “ab” o “a” seguido de cualquier número de “b”.

  ?: Hace que la RE resultante coincida con 0 o 1 repeticiones de la RE precedente. ab? coincidirá con “a” o “ab”.

  +: Hace que la RE resultante coincida con 1 o más repeticiones de la RE precedente. ab+ coincidirá con “a” seguido de cualquier número distinto de cero de “b”; no coincidirá solo con “a”.

  [ ]: Se utiliza para indicar un conjunto de caracteres

También se utiliza [string](https://docs.python.org/es/3/library/string.html) que tiene metodos que permiten acceder a distintos tipos de caracteres. Un ejemplo es acceder a los signos de puntuacion con `string.punctuation`

In [ ]:
# CARACTERES QUE NECESITAMOS ELIMINAR
print(string.punctuation) # Devuelve todos los caracteres de puntuación comunes como una cadena.
print(re.escape(string.punctuation)) # La función escape() del módulo re se utiliza para "escapar" todos los caracteres especiales de string.punctuation.

In [ ]:
# DEFINIMOS UNA FUNCION DE LIMPIEZA
# LA FUNCION RECIBE UN TEXTO Y DEVUELVE EL MISMO TEXTO SIN SIGNOS
def clean_text_round1(text):
    # CONVERTIMOS TODO A MINÚSCULAS.
    text = text.lower()
    # REEMPLAZAMOS TEXTO ENTRE CORCHETES POR ESPACIOS EN BLANCO.
    text = re.sub('\[.*?¿\]\%', ' ', text)
    # REEMPLAZAMOS SIGNOS DE PUNTUACION POR ESPACIOS EN BLANCO (%s -> \S+ es cualquier caracter que no sea un espacio en blanco).
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # REMUEVE PALABRAS QUE CONTIENEN NUMEROS.
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
# Defino una funcion anonima que al pasarle un argumento devuelve el resultado de aplicarle la funcion anterior a este mismo argumento
round1 = lambda x: clean_text_round1(x)

# Dataframe que resulta de aplicarle a las columnas la funcion de limpieza
data_clean = pd.DataFrame(data_df.transcript.apply(round1))

In [ ]:
print(data_clean)

In [ ]:
data_clean.transcript[0]

In [ ]:
# DEFINIMOS OTRA FUNCION DE LIMPIEZA
# LA FUNCION RECIBE UN TEXTO Y DEVUELVE EL MISMO TEXTO SIN COMILLAS, PUNTOS SUSPENSIVOS, <<, >>
def clean_text_round2(text):
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

In [ ]:
round2 = lambda x: clean_text_round2(x)

data_clean = pd.DataFrame(data_clean.transcript.apply(round2))

In [ ]:
# SERIALIZAMOS EL RESULTADO Y GUARDAMOS EL CORPUS FINAL A TRATAR
# data_df.to_pickle("/content/drive/MyDrive/Clases/HABLA/003/PRA/cuentos_casciari/corpus.pkl")
data_df.to_pickle("C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\corpus.pkl")

In [ ]:
print(data_clean)

In [ ]:
data_clean.transcript[0]

### 3.- CREAMOS EL “BAG OF WORDS”

In [ ]:
# IMPORTAMOS LAS LIBRERIAS
# CountVectorizer: Convierte un conjunto de textos en vectores numéricos, donde cada columna representa una palabra del vocabulario y cada fila 
# representa un texto. Es lo que se llama una bolsa de palabras (bag of words): solo cuenta cuántas veces aparece cada palabra en el texto 
# (sin importar el orden).
from sklearn.feature_extraction.text import CountVectorizer # CREAMOS LA MATRIZ DE PALABRAS
# NLTK es una libreria que ofrece herramientas para:
# * Tokenización (dividir texto en palabras, frases, oraciones)
# * Lematización y stemming
# * Etiquetado gramatical (POS tagging)
# * Extracción de entidades
# * Corpus integrados (textos de ejemplo)
# *Modelos de lenguaje sencillos
import nltk

In [ ]:
nltk.download('stopwords') # DESCARGAMOS EL MODULO "stopwords"
lines = nltk.corpus.stopwords.words('spanish') # ASIGNAMOS LAS "stopwords" DEL ESPAÑOL

In [ ]:
# VISUALIZAMOS EL TIPO DE VARIABLE (LISTA) Y CANTIDAD DE PALABRAS
print(type(lines))
len(lines)

In [ ]:
# VERIFICAMOS QUE CONTAMOS CON LAS STOPWORDS DEL ESPAÑOL
print(lines)

Ahora vamos a crear la matriz de terminos-documentos. Para ello hay una funcion de scikit-learn que si le das una secuencia de items de tipo string o byte hace exactamente eso.

In [ ]:
# INSTANCIAMOS UNA VARIABLE DE TIPO "CountVectorizer" Y LE INDICAMOS QUE CUENTA AHORA CON UN "stop_words" DEFINIDO
count_vect = CountVectorizer(stop_words=lines)

In [ ]:
# TRANSFORMAMOS EL TEXTO EN DATOS NUMÉRICOS USANDO "CountVectorizer" PARA CONVERTIR LOS TEXTOS DE "data_clean.transcript" EN UNA MATRIZ.
data_cv = count_vect.fit_transform(data_clean.transcript)

In [ ]:
# CREAMOS UN DATAFRAME TRANSFORMANDO CON LA MATRIZ QUE GENERAMOS ANTES DONDE:
# 1.- LAS COLUMNAS SON LAS PALABRAS
# 2.- LAS FILAS SON LOS DOCUMENTOS
data_dtm = pd.DataFrame(data_cv.toarray(), columns=count_vect.get_feature_names_out())

In [ ]:
# ASIGNAMOS LOS INDICES DEL DATAFRAME ANTERIOR
data_dtm.index = data_clean.index

En el contexto del procesamiento de texto y el análisis de datos, la matriz esparsa es una herramienta fundamental. Esta matriz surge del deseo de representar documentos como vectores, donde cada dimensión (o columna) se corresponde con una palabra del vocabulario.

La razón por la que utilizamos matrices esparsas es que en la mayoría de los casos, un documento contiene solo un pequeño subconjunto del vocabulario total. Por ejemplo, si tenemos un corpus con miles de palabras pero un documento particular solo utiliza unas pocas, la matriz resultante de la representación será mayormente llena de ceros.

Ejemplo:

Supongamos que tenemos 3 documentos y un vocabulario de 4 palabras:

Documentos:

* 1: "manzana"
* 2: "banana"
* 3: "manzana banana"

Vocabulario: ["manzana", "banana", "uva", "pera"]

La matriz resultante sería:

        manzana  banana  uva  pera
     
     1       1       0    0     0   # Documento 1
     
     2       0       1    0     0   # Documento 2
     
     3       1       1    0     0   # Documento 3


Esta representación eficiente tiene varias ventajas:

1. **Ahorro de espacio**: Almacenar solo los valores no cero y sus posiciones ahorra considerablemente espacio en disco y memoria.

2. **Rendimiento**: Muchas operaciones matemáticas en matrices esparsas se pueden hacer de manera más rápida y eficiente, lo que es crucial para manejar grandes volúmenes de texto.

Como resultado, al ajustar el modelo (como `CountVectorizer` o `TF-IDF`), generamos una matriz esparsa que nos permite realizar análisis más complejos sin perder rendimiento ni eficiencia.

In [ ]:
# VISUALIZAMOS LA DEFINICIÓN DE LA MATRIZ ESPARZA/DISPERSA: ES UNA MATRIZ QUE CONTIENE MUCHISIMOS CEROS Y MUY POCOS VALORES DISTINTO DE CERO.
data_cv

In [ ]:
# VISUALIZAMOS LA MATRIZ EN FORMATO DATAFRAME (12 FILAS Y 29713 COLUMNAS)
data_dtm

----
Vemos que hay palabras que no estan en el idioma espaniol como **českomoravský**. Dependiendo la informacion que queramos obtener con estos datos entonces tal vez es conveniente filtrar este tipo de palabras. Como lo que vamos a analizar en esta notebook es el vocabulario del escritor, en particular el numero de palabras unicas, entonces las vamos a dejar.

**nltk** tiene [corpus](https://www.nltk.org/book/ch02.html) que se pueden cargar y utilizar para filtrar palabras de distintos idiomas o nombres propios.

```
# Ejemplo de palabras en ingles
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
```

Ojo! que hay palabras que estan en ingles pero no estan en un *diccionario*, como expresiones tipicas de chats, algunos nombres propios o abreviaturas. Entonces hay que tener cuidado cuando pasamos de mayusculas a minusculas. Si las queremos filtrar entonces tenemos que ser cuidadosxs con el corpus que usamos.

---

### 4.- GUARDAMOS TODOS LOS DATOS GENERADOS (SERIALIZADO)

In [ ]:
# GUARDAMOS EL DATAFRAME
data_dtm.to_pickle("C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\dtm.pkl")

In [ ]:
# GURDAMOS EL TEXTO LIMPIO SOBRE EL CUAL TRABAJAMOS
data_clean.to_pickle('C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\data_clean.pkl')

In [ ]:
# GUARDAMOS EL OBJETO CountVectorize
pickle.dump(count_vect, open("C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\cv.pkl", "wb"))

In [ ]:
count_vect

### 5.- ANÁLISIS EXPLORATORIO

#### 5.1 CARGAMOS EL DATAFRAME QUE GUARDAMOS ANTERIORMENTE

In [ ]:
# data = pd.read_pickle('/content/drive/MyDrive/Clases/HABLA/003/PRA/cuentos_casciari/dtm.pkl')
data = pd.read_pickle('C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\dtm.pkl')

In [ ]:
# TRANSPONEMOS LA MATRIZ PARA TENER UNA EN FORMATO "terminosXdocumentos"
data = data.transpose()

In [ ]:
# MOSTRAMOS LOS PRIMEROS 10 REGISTROS
data.head(10)

#### 5.2 BUSCAMOS LAS PALABRAS MAS USADAS POR AÑO

In [ ]:
# CREAMOS UN DICCIONARIO VACÍO
top_dict = {}

In [ ]:
# RECORRO LA MATRIZ POR CADA COLUMNA (AÑO)
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(30) # Ordeno las filas en forma decreciente y me quedo con las 30 palabras mas usadas
    top_dict[c]= list(zip(top.index, top.values))       # le asigno el año a la key del diccionario y como valor una tupla con la palabra y su frecuencia

In [ ]:
print(top_dict)
print("\n")

In [ ]:
# MOSTRAMOS LAS 15 PALABRAS MAS FRECUENTES POR CADA AÑO
for anio, top_words in top_dict.items():
    print(anio) # IMPRIMO LAS KEYS
    print(', '.join([word for word, count in top_words[0:14]])) # MOSTRAMOS LAS PALABRAS EN ORDEN DECRECIENTE SEGUN FRECUENCIA Y SEPARADAS CON ESPACIO Y COMA.

#### 5.3 AGREGAMOS LAS STOP_WORDS

Vemos que hay nombres propios, numeros escrito con letras y algunas palabras como el adverbio "tan", "si", "cada", etc. que no nos da informacion por si solo. Debemos filtrar estas palabras.

Tambien vemos que hay palabras que se repiten dentro de las mas frecuentes de cada anio en todos los *anios* que tambien queremos filtrar

En este caso vamos a unir las 12 listas de palabras mas usadas en un nuevo ranking y de esas, tomaremos las "mas usadas" para agregar al listado de Stop Words. Es decir, si una palabra aparecio en el top 30 mas de la mitad de las veces (en 7 anios o mas) entonces la considero una stop word.

Vamos a usar la funcion Counter de la libreria collections que dada una lista de strings nos devuelve la cantidad de repeticiones de cada elemento

In [ ]:
# IMPORTAMOS LA LIBRERIA
from collections import Counter

In [ ]:
# GENERAMOS UNA LISTA CON LAS 30 PALABRAS MAS USADAS DE CADA AÑO
words = []
for anio in data.columns:
    top = [word for (word, count) in top_dict[anio]] # armo una lista quedandome solo con las palabras del diccionario que creamos antes
    for t in top:
        words.append(t)                              # agrego cada palabra a la lista vacia, por separado

In [ ]:
# usando el metodo .most_common() obtengo las palabras mas usadas, y la cantidad de veces que fue agregada a la lista words
print(Counter(words).most_common())

In [ ]:
# Creo una lista de nuevas stop words considerando como stop words a palabras que recibieron mas de la mitad de las cuentas posibles (12 anios)
add_stop_words = [word for word, count in Counter(words).most_common() if count > 6]
print(add_stop_words)

#### 5.4 ACTUALIZAMOS LA "BAG OF WORDS"

Ahora quitaremos las "stop_words" de nuestro dataset. Para esto hacemos lo siguiente:

1.- Usamos el listado de "stop_words" en espaniol de nltk, el que generamos recién y uno adicional que tiene en cuenta las observaciones que palabras fueron las mas frecuentes.

2.- Luego, volveremos a cargar los datos limpios y la filtramos aplicando las dos nuevas listas.

In [ ]:
# IMPORTAMOS LA LIBRERIA
from sklearn.feature_extraction import text

In [ ]:
# RECUPERAMOS LOS DATOS LIMPIOS QUE GUARDAMOS EN LA SERIALIZACIÓN
data_clean = pd.read_pickle('C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\data_clean.pkl')

In [ ]:
type(data_clean)

In [ ]:
data_clean

In [ ]:
# Agregamos la lista de stop words "lines" las nuevas stop words
for pal in add_stop_words:
    lines.append(pal)

In [ ]:
type(lines)

In [ ]:
lines

In [ ]:
len(lines)

In [ ]:
# Creamos la lista que tiene algunos nombres propios que aparecieron entre las palabras mas frecuentes y otro tipo de palabras
more_stop_words=['alex','lucas','andrés','mirta','tres','primer','primera','dos','uno','veces', 'así', 'luego', 'quizá','cosa','cosas','tan','asi','andres','todas','sólo','jesús','pablo','pepe']
for pal in more_stop_words:
    lines.append(pal)

In [ ]:
lines

In [ ]:
len(lines)

In [ ]:
# Recreamos la matriz de documentos y terminos pero ustando la nueva lista "mejorada" de stopwords
count_vect = CountVectorizer(stop_words=lines)

In [ ]:
data_cv = count_vect.fit_transform(data_clean.transcript)

In [ ]:
# Use get_feature_names_out() instead of get_feature_names()
data_stop = pd.DataFrame(data_cv.toarray(), columns=count_vect.get_feature_names_out())
data_stop.index = data_clean.index

In [ ]:
# Lo guardamos en formato pickle
pickle.dump(count_vect, open("C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\cv_stop.pkl", "wb"))

In [ ]:
data_stop.to_pickle("C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\dtm_stop.pkl")

#### 5.5 ARMAMOS LA "NUBE DE PALABRAS"
Queremos ver “que tenia Hernan Casciari en su cabeza” entre 2004 y 2015 en sus cuentos usando WordClouds.

In [ ]:
# IMPORTAMOS LAS LIBRERIAS
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Creo el objeto WordCloud con determinados parametros y utilizando nuestra lista de stopwords
wc = WordCloud(stopwords=lines, background_color="white", colormap="Dark2", max_font_size=150, random_state=42)

In [ ]:
plt.rcParams['figure.figsize'] = [16,12] # tamanio de los plots

In [ ]:
# Para cada anio creo un WC
for index, anio in enumerate(data.columns):
    wc.generate(data_clean.transcript[anio])  # aca le pido que genere los WC a partir del texto de cada anio
    plt.subplot(4, 3, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(anios[index])
plt.show()

La hija de Hernan Casciari nacio en el 2004. Se puede tratar de analizar como el nacimiento de su hija influencio su escritura. En el 2007, cuando ella tenia tres anios, vemos que empezo a tener mas importancia la palabra "padre" en sus cuentos. Y cuando ella tenia once anios, en el 2015, es la palabra mas frecuente. Siguiendo este analisis se puede pensar que verla crecer afecto enormemente el contenido de sus cuentos.

Siguiendo este analisis interpretativo se nota en estas wordclouds algunos topicos importantes que definen su personalidad. Es un escritor que le gusta mucho escribir sobre su vida, por lo que no es de extraniarse que "libro", "revista" u "orsai", su editorial, aparezcan en muchas WCs. Tambien sabemos que es un fanatico del "futbol", y que es muy familiero ("padre", "madre", "hija")


#### 5.6 ESTADÍSTICAS DE PALABRAS POR AÑO
Ahora sacaremos algunas estadísticas de palabras únicas por año (el tamaño del vocabulario empleado) y el promedio de palabras por artículo

In [ ]:
#
data[data.columns[1]].to_numpy

In [ ]:
# Encontraremos las palabras unicas por anio identificando los elementos non-zero en la matriz de documentos y terminos
unique_list = []
for anio in data.columns:
    uniques = data[anio].to_numpy().nonzero()[0].size # tengo que transformar a un array para aplicar la funcion nonzero
    unique_list.append(uniques)

In [ ]:
# Creo un nuevo dataframe con el numero de palabras unicas por anio
data_words = pd.DataFrame(list(zip(anios, unique_list)), columns=['Anio', 'unique_words'])

In [ ]:
#data_unique_sort = data_words.sort_values(by='unique_words')
data_unique_sort = data_words # sin ordenar
print(data_unique_sort)

In [ ]:
# El numero de post por anio, esto es dato
posts_per_year = [50, 27, 18, 50, 42, 22, 50, 33, 31, 17, 33, 13]

In [ ]:
# Encuentro el numero total de palabras por anio
total_list = []
for anio in data.columns:
    totals = sum(data[anio])
    total_list.append(totals)

In [ ]:
# Agrego las columnas al dataframe
data_words['total_words'] = total_list
data_words['posts_per_year'] = posts_per_year
data_words['words_per_posts'] = data_words['total_words'] / data_words['posts_per_year']

data_wpm_sort = data_words #sin ordenar
print(data_wpm_sort)

#### 5.7 VISUALIZACIÓN DE LA TABLA
Veamos los datos en gráfico de barras horizontales:

In [ ]:
import numpy as np
plt.rcParams['figure.figsize'] = [16, 6]

y_pos = np.arange(len(data_words))

plt.subplot(1, 3, 1)
plt.barh(y_pos,posts_per_year, align='center')
plt.yticks(y_pos, anios)
plt.title('# de Posts', fontsize=20)

plt.subplot(1, 3, 2)
plt.barh(y_pos, data_unique_sort.unique_words, align='center')
plt.yticks(y_pos, data_unique_sort.Anio)
plt.title('# palabras unicas', fontsize=20)

plt.subplot(1, 3, 3)
plt.barh(y_pos, data_wpm_sort.words_per_posts, align='center')
plt.yticks(y_pos, data_wpm_sort.Anio)
plt.title('# palabras por post', fontsize=20)

plt.tight_layout()
plt.show()

Y hagamos una comparativa de frecuencia de uso de algunas palabras (se pueden elegir cualquiera)

In [ ]:
import nltk
from nltk.corpus import PlaintextCorpusReader

corpus_root = "C:\\Users\\ddulcich\\Documents\\PERSONAL\\IFTS24\\16.-TECNICAS DE PROCESAMIENTO DE HABLA\\TPS\\libros\\"
wordlists = PlaintextCorpusReader(corpus_root, '.*', encoding='latin-1')
#wordlists.fileids() # con esto listamos los archivos del directorio

cfd = nltk.ConditionalFreqDist(
        (word,genre)
        for genre in anios
        for w in wordlists.words(genre + '.txt')
        for word in ['casa','futbol','tiempo','vida']
        if w.lower().startswith(word) )
cfd.plot()

## Conclusiones

Repasemos lo que hicimos y que resultados sacamos:

* Limpiamos los textos, quitamos caracteres que no utilizamos y creamos un listado de stop_words (palabras para omitir)

* Exploración de datos:
Realizamos estadísticas básicas, como el vocabulario usado, cantidad de palabras por año y promedio por posts.

* Creamos Nubes de Palabras por año ya que es una manera de visualizar textos y frecuencia de termino a lo largo de los anios (similar a [Google Ngrams Viewer](https://books.google.com/ngrams)) falta dividir por la cantidad total de palabras de cada anio